# Script 9: Pulizia Speed e Imputazione Skills (Fixed)

In [22]:
import pandas as pd
import numpy as np

# Percorso del file
input_path = "./data/monsters.csv" 

# Caricamento
try:
    df = pd.read_csv(input_path)
    print(f"✅ Dataset caricato: {input_path} ({df.shape})")
except FileNotFoundError:
    df = pd.read_csv("monsters.csv")
    print(f"✅ Dataset caricato (root): monsters.csv ({df.shape})")


✅ Dataset caricato: ./data/monsters.csv ((2435, 62))


In [23]:
# 1. Pulizia nomi colonne
df.columns = (
    df.columns
    .str.strip()
    .str.replace("!", "", regex=False)
)


## Gestione Feature "Speed" speciali


In [24]:
# Rimozione colonne speed inutili
cols_to_remove = ['speed.lightwalking', 'speed.bur.']
cols_existing_drop = [c for c in cols_to_remove if c in df.columns]

if cols_existing_drop:
    df = df.drop(columns=cols_existing_drop)
    print(f"✅ Colonne rimosse: {cols_existing_drop}")

# Gestione speed.hover (Fix Warning)
if 'speed.hover' in df.columns:
    # Sostituiamo i NaN con 0
    df['speed.hover'] = df['speed.hover'].fillna(0)
    
    # Logica robusta: Convertiamo tutto in stringa e verifichiamo se contiene "True" o "1"
    # Questo evita problemi di tipi misti (booleani, stringhe, float) e il FutureWarning di replace
    df['speed.hover'] = np.where(
        df['speed.hover'].astype(str).str.contains("True|1", case=False, na=False), 
        1, 
        0
    )
    
    print("✅ speed.hover convertita in binario (0/1).")


✅ Colonne rimosse: ['speed.lightwalking']
✅ speed.hover convertita in binario (0/1).


## Imputazione Skill (Metodo D&D Rules)
Formula: `Mod = (Score - 10) // 2`


In [25]:
# 1. Pre-imputazione zeri per le skill (solo colonne numeriche)
start_column = "speed.walk"
if start_column in df.columns:
    start_idx = df.columns.get_loc(start_column)
    cols_to_fill = df.columns[start_idx:]
    # Seleziona solo colonne numeriche per evitare errori
    num_cols = df[cols_to_fill].select_dtypes(include=[np.number]).columns
    df[num_cols] = df[num_cols].fillna(0)

# Mappa Skill -> Caratteristica
skill_to_ability = {
    "skills.athletics": "strength",
    "skills.acrobatics": "dexterity",
    "skills.stealth": "dexterity",
    "skills.sleight_of_hand": "dexterity",
    "skills.arcana": "intelligence",
    "skills.history": "intelligence",
    "skills.investigation": "intelligence",
    "skills.nature": "intelligence",
    "skills.religion": "intelligence",
    "skills.insight": "wisdom",
    "skills.medicine": "wisdom",
    "skills.perception": "wisdom",
    "skills.survival": "wisdom",
    "skills.animal_handling": "wisdom",
    "skills.deception": "charisma",
    "skills.intimidation": "charisma",
    "skills.performance": "charisma",
    "skills.persuasion": "charisma",
}

print("Inizio imputazione skill...")

count_imputed = 0
for skill_col, ability_col in skill_to_ability.items():
    if skill_col in df.columns and ability_col in df.columns:
        
        # Calcolo modificatore
        modifier = (df[ability_col] - 10) // 2
        
        # Identifica dove la skill è 0 (o NaN convertito in 0)
        mask_zero = (df[skill_col] == 0)
        
        # Assegnazione
        if mask_zero.sum() > 0:
            df.loc[mask_zero, skill_col] = modifier[mask_zero]
            count_imputed += mask_zero.sum()
        
        # Casting finale a int
        df[skill_col] = df[skill_col].astype(int)

print(f"✅ Imputazione completata. Valori ricalcolati: {count_imputed}")


Inizio imputazione skill...
✅ Imputazione completata. Valori ricalcolati: 36308


In [26]:
# Output e Verifica
output_path = "./data/monsters.csv"

try:
    df.to_csv(output_path, index=False)
    print(f"\n💾 File salvato con successo: {output_path}")
    
    # Verifica
    check_cols = ["name", "speed.hover", "dexterity", "skills.stealth"]
    print(df[check_cols].head(5))
except PermissionError:
    print("\n❌ ERRORE: File aperto. Chiudilo e riprova.")



💾 File salvato con successo: ./data/monsters.csv
                 name  speed.hover  dexterity  skills.stealth
0             Aboleth            0          9              -1
1             Acolyte            0         10               0
2  Adult Black Dragon            0         14               7
3   Adult Blue Dragon            0         10               5
4  Adult Brass Dragon            0         10               5
